# Lesson 18 - Filtering, Sorting, and Grouping

In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, expr


spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext

## Introduction

In this lesson, we will introduce DataFrame methods that can be used to perform tasks commonly required when working with tabular data. Specifically, we will discuss how to filter DataFrames, how to sort the rows of a DataFrame, and how to perform grouping and aggregation on the rows of a DataFrame. We will use the Gapminder dataset to illustrate these concepts.

In [0]:
gm_schema = (
    'country STRING, year INTEGER, continent STRING, population INTEGER, '
    'life_exp DOUBLE, gdp_per_cap DOUBLE, gini DOUBLE'
)

gm_df = (
  spark.read
  .option('delimiter', '\t')
  .option('header', True)
  .schema(gm_schema)
  .csv('/FileStore/tables/gapminder_data_2020.txt')
)
    
gm_df.printSchema()

root
-- country: string (nullable = true)
-- year: integer (nullable = true)
-- continent: string (nullable = true)
-- population: integer (nullable = true)
-- life_exp: double (nullable = true)
-- gdp_per_cap: double (nullable = true)
-- gini: double (nullable = true)

### The `filter()` Transformation

We can use the `filter()` transformation to reduce the size of a DataFrame by selecting out rows that satisfy a certain condition. We can specify the condition by applying a Python comparison operator to a column object, or by specifying the desired comparison as an expression string provided to `expr()`.

In [0]:
#gm_20 = gm_df.filter(col('year') == 2020)
gm_20 = gm_df.filter(expr('year == 2020'))
gm_20.persist()
gm_20.show(10)

+-------------------+----+---------+----------+--------+-----------+----+
 country|year|continent|population|life_exp|gdp_per_cap|gini|
+-------------------+----+---------+----------+--------+-----------+----+
 Afghanistan|2020| asia| 38900000| 64.4| 1800.0|36.8|
 Albania|2020| europe| 2880000| 78.6| 13200.0|29.0|
 Algeria|2020| africa| 43900000| 78.3| 14000.0|27.6|
 Angola|2020| africa| 32900000| 65.4| 5440.0|42.6|
Antigua and Barbuda|2020| americas| 97900| 77.4| 25000.0|40.0|
 Argentina|2020| americas| 45200000| 77.1| 17100.0|42.4|
 Armenia|2020| europe| 2960000| 76.0| 10200.0|32.6|
 Australia|2020| asia| 25500000| 82.8| 45800.0|32.3|
 Austria|2020| europe| 9010000| 82.1| 47300.0|30.5|
 Azerbaijan|2020| europe| 10100000| 71.2| 16400.0|32.4|
+-------------------+----+---------+----------+--------+-----------+----+
only showing top 10 rows

We apply more complicated filters by chaining calls to `filter()` or by using logical operators such as `AND` and `OR` inside our expression strings.

In [0]:
gm_df.filter(expr('year == 2020 AND continent == "asia"')).count()

Out[4]: 53

### The `sort()` Transformation 

We can use the `sort()` transformation to sort a DataFrame by one or more columns. This transformation accepts one or more arguments, each of which is expected to be either a column object or a string containing the name of a column. In the cell below, we use `sort()` to identify the five countries with the lowest life expectency in 2020.

In [0]:
gm_20.sort('life_exp').show(5, truncate=False)

+------------------------+----+---------+----------+--------+-----------+----+
country |year|continent|population|life_exp|gdp_per_cap|gini|
+------------------------+----+---------+----------+--------+-----------+----+
Central African Republic|2020|africa |4830000 |53.3 |817.0 |56.2|
Lesotho |2020|africa |2140000 |56.6 |2900.0 |54.2|
Papua New Guinea |2020|asia |8950000 |59.1 |3950.0 |41.9|
Somalia |2020|africa |15900000 |59.5 |633.0 |48.0|
South Sudan |2020|africa |11200000 |60.0 |1950.0 |45.0|
+------------------------+----+---------+----------+--------+-----------+----+
only showing top 5 rows

The `sort()` method has an optional `ascending` parameter. This is equal to `True` by default, but can be set `False` to sort the DataFrame rows in descending order.

In [0]:
gm_20.sort('life_exp', ascending=False).show(5)

+-----------+----+---------+----------+--------+-----------+----+
 country|year|continent|population|life_exp|gdp_per_cap|gini|
+-----------+----+---------+----------+--------+-----------+----+
 Singapore|2020| asia| 5850000| 85.3| 90500.0|40.9|
 Japan|2020| asia| 126000000| 84.7| 40100.0|32.1|
Switzerland|2020| europe| 8650000| 84.4| 59100.0|32.3|
 Italy|2020| europe| 60500000| 83.6| 36100.0|35.4|
 Spain|2020| europe| 46800000| 83.5| 36200.0|36.2|
+-----------+----+---------+----------+--------+-----------+----+
only showing top 5 rows

We can sort by more than one column at a time by indicating each of the columns to be involved in the sort. The first column provided is the primary column according to which the results are sorted.

In [0]:
gm_20.sort('continent', 'life_exp').show(5)

+--------------------+----+---------+----------+--------+-----------+----+
 country|year|continent|population|life_exp|gdp_per_cap|gini|
+--------------------+----+---------+----------+--------+-----------+----+
Central African R...|2020| africa| 4830000| 53.3| 817.0|56.2|
 Lesotho|2020| africa| 2140000| 56.6| 2900.0|54.2|
 Somalia|2020| africa| 15900000| 59.5| 633.0|48.0|
 South Sudan|2020| africa| 11200000| 60.0| 1950.0|45.0|
 Mozambique|2020| africa| 31300000| 60.4| 1210.0|54.0|
+--------------------+----+---------+----------+--------+-----------+----+
only showing top 5 rows

We can also specify that a column is to be sorted in descending order by calling the `desc()` method of an associated column object. This can be useful if we want to sort by multiple columns, but want to sort some of the columns in ascending order and others in descending order.

In [0]:
gm_20.sort(
    col('continent').desc(), 
    col('life_exp')
).show(5)

+----------+----+---------+----------+--------+-----------+----+
 country|year|continent|population|life_exp|gdp_per_cap|gini|
+----------+----+---------+----------+--------+-----------+----+
 Ukraine|2020| europe| 43700000| 70.9| 8770.0|25.0|
Azerbaijan|2020| europe| 10100000| 71.2| 16400.0|32.4|
 Russia|2020| europe| 146000000| 72.7| 26200.0|37.7|
 Moldova|2020| europe| 4030000| 73.2| 6980.0|26.4|
 Georgia|2020| europe| 3990000| 73.3| 11300.0|36.5|
+----------+----+---------+----------+--------+-----------+----+
only showing top 5 rows

We can also sort by functions of column values, rather than the column values themselves. In the cell below, we sort the `gm_20` DataFrame in decreasing order of length of the name of the countries.

In [0]:
gm_20.sort(expr('LENGTH(country)'), ascending=False).show(10, truncate=False)

+------------------------------+----+---------+----------+--------+-----------+----+
country |year|continent|population|life_exp|gdp_per_cap|gini|
+------------------------------+----+---------+----------+--------+-----------+----+
St. Vincent and the Grenadines|2020|americas |111000 |72.7 |11400.0 |40.0|
Central African Republic |2020|africa |4830000 |53.3 |817.0 |56.2|
Bosnia and Herzegovina |2020|europe |3280000 |77.1 |13500.0 |32.7|
Micronesia, Fed. Sts. |2020|asia |115000 |67.6 |3260.0 |40.1|
Sao Tome and Principe |2020|africa |219000 |67.5 |3090.0 |30.8|
United Arab Emirates |2020|asia |9890000 |73.8 |65300.0 |40.0|
Trinidad and Tobago |2020|americas |1400000 |74.6 |28800.0 |41.3|
Antigua and Barbuda |2020|americas |97900 |77.4 |25000.0 |40.0|
Dominican Republic |2020|americas |10800000 |73.7 |17100.0 |45.3|
Equatorial Guinea |2020|africa |1400000 |66.6 |17700.0 |40.0|
+------------------------------+----+---------+----------+--------+-----------+----+
only showing top 10 rows

### The `groupBy()` Transformation

Spark provides tools for performing grouping and aggregation. These tools allow us to group together rows of a DataFrame that share the same values in one or more columns, and then report summary statistics for each such group. This process is performed in two steps:

1. Call the `groupBy()` method of the DataFrame, passing it one or more column objects or names of columns. This specific the column or columns upon which to group.
2. Call the `agg()` method of the grouped DataFrame, passing it one or more expressions defining aggregations to be performed on columns of the DataFrame.

The resulting DataFrame will contain one row for each unique value within the column that we grouped on. It will contain on column indicating these unique grouped, and one column for each aggregation we defined.

In the cell below, we group the 2020 data according to continental region and then calculate the total population for each such region.

In [0]:
( 
    gm_20
    .groupBy('continent')
    .agg(expr('SUM(population) AS total_pop'))
    .sort('total_pop')
    .show()
)

+---------+----------+
continent| total_pop|
+---------+----------+
 europe| 849707000|
 americas|1018327900|
 africa|1338451300|
 asia|4549608000|
+---------+----------+

We can perform as many aggregations as we would like within a single call to `agg()`. Each aggregation will define a new column in the grouped DataFrame.

In [0]:
( 
    gm_20
    .groupBy('continent')
    .agg(
        expr('COUNT(*) AS n_countries'),
        expr('SUM(population) AS total_pop'),
        expr('INT(MEAN(population)) AS avg_pop'),
        expr('MIN(population) AS min_pop'),
        expr('MAX(population) AS max_pop')     
    )
    .sort('continent')
    .show()
)

## Example: Life Expectency and Per Capita GDP

We will close this lesson with an example in which we calculate the total population, life expectency, and per capita GDP for each of the four continental regions in the Gapminder dataset.

In [0]:
( 
    gm_20
    .select(
        '*',
        expr('gdp_per_cap * population as total_gdp'),
        expr('life_exp * population as pop_wt_life_exp')
    )
    .groupBy('continent')
    .agg(
        expr('COUNT(*) AS n_countries'),
        expr('SUM(population) AS total_pop'),
        expr('SUM(total_gdp) AS total_gdp'),
        expr('SUM(pop_wt_life_exp) AS pop_wt_life_exp')
    )
    .select(
        'continent', 'n_countries', 'total_pop',
        expr('INT(total_gdp / total_pop) AS gdp_per_cap'),
        expr('ROUND(pop_wt_life_exp / total_pop,1) AS life_exp'),
    )
    .sort('continent')
    .show()
)

+---------+-----------+----------+-----------+--------+
continent|n_countries| total_pop|gdp_per_cap|life_exp|
+---------+-----------+----------+-----------+--------+
 africa| 54|1338451300| 4805| 66.9|
 americas| 33|1018327900| 29499| 77.4|
 asia| 53|4549608000| 13710| 73.8|
 europe| 44| 849707000| 33034| 78.8|
+---------+-----------+----------+-----------+--------+